### ATTENTION - Restatement of flow and code is under progress. 

####### little massy right now

In [0]:
%pip install 'protobuf<=3.20.1' --force-reinstall --quiet

In [0]:
from poirot.azure.databricks import svc_prncpl_magic_setup
svc_prncpl_magic_setup(scope="kv-8451-tm-media-dev", app_id_secret="spTmMediaDev-app-id", app_pw_secret="spTmMediaDev-pw")


In [0]:
#Define service principals
service_credential = dbutils.secrets.get(scope='kv-8451-tm-media-dev',key='spTmMediaDev-pw')
service_application_id = dbutils.secrets.get(scope='kv-8451-tm-media-dev',key='spTmMediaDev-app-id')
directory_id = "5f9dc6bd-f38a-454a-864c-c803691193c5"
storage_account = 'sa8451dbxadhocprd'

In [0]:
spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{storage_account}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{storage_account}.dfs.core.windows.net", service_application_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{storage_account}.dfs.core.windows.net", service_credential)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{storage_account}.dfs.core.windows.net", f"https://login.microsoftonline.com/{directory_id}/oauth2/token")

In [0]:
#Importing All the libraries
import pandas as pd
import numpy as np
import kayday as kd
import seg
import toolbox.config as tox_con
import pyspark.sql.functions as f
import pyspark.sql.types as t
import datetime as dt
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from functools import reduce
from datetime import datetime, timedelta
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.types import *
from effodata import ACDS, golden_rules, Sifter, Equality, Joiner
from toolbox.config import segmentation
from kpi_metrics import KPI, AliasMetric, CustomMetric, AliasGroupby, available_metrics, get_metrics
from seg.utils import DateType
from flowcate.files import FilePath
from pyspark.sql.functions import col, udf
from pyspark.sql.types import BooleanType
from pyspark.dbutils import DBUtils

In [0]:
spark.conf.set("spark.sql.sources.commitProtocolClass", "org.apache.spark.sql.execution.datasources.SQLHadoopMapReduceCommitProtocol")
spark.conf.set('parquet.enable.summary-metadata', 'false')
spark.conf.set('mapreduce.fileoutputcommitter.marksuccessfuljobs', 'false')
spark.conf.set("spark.databricks.queryWatchdog.maxQueryTasks", "400000")
# spark.conf.set("spark.sql.shuffle.partitions", "50")

In [0]:

timestamp = "abfss://media@sa8451dbxadhocprd.dfs.core.windows.net/audience_factory/egress_receipts/"
eligibility_path = ('abfss://landingzone@sa8451entlakegrnprd.dfs.core.windows.net/mart/comms/prd/fact/eligibility_fact')
hh_location = "abfss://media@sa8451dbxadhocprd.dfs.core.windows.net/audience_factory/powerbi_inputs/household_counts/"
elig_file = "abfss://media@sa8451dbxadhocprd.dfs.core.windows.net/audience_factory/powerbi_inputs/eligibility_counts/"
image_location =  "abfss://media@sa8451dbxadhocprd.dfs.core.windows.net/audience_factory/powerbi_inputs/valid_image_upcs"
image_upc = "abfss://media@sa8451dbxadhocprd.dfs.core.windows.net/audience_factory/powerbi_inputs/upc_via_segmentation/"

filtered_time_period_lst = spark.read.parquet(hh_location).select("PERIOD").distinct().rdd.flatMap(lambda x: x).collect()
final_per_list = ', '.join(map(str, filtered_time_period_lst))

files_path = dbutils.fs.ls(eligibility_path)

sorted_files = sorted(files_path, key=lambda x: x[1], reverse=True)

# Get the second latest file
second_latest_file = sorted_files[1]

print("The latest file is: ", second_latest_file.path)

latest_eligibility_df = spark.read.parquet(second_latest_file.path) #This will give us the Dates we are using in. #Getting Eligiblity Table


In [0]:
def get_all_segmentation_name():
    '''
    Get all the Segmentation names
    
    '''
    segmentation_names = tox_con.segmentations.all_segmentations

    return segmentation_names

In [0]:
dbutils = DBUtils()
needs_to_measured = []
dir_contents = dbutils.fs.ls(timestamp)
dir_contents = [ x[0] for x in dir_contents]

segmentation_name = get_all_segmentation_name()

for i in range(0, len(dir_contents)):
  if dir_contents[i][-12:-4] in final_per_list:
    print(f'Period already exist {dir_contents[i][-12:-4]}')
  else:
    needs_to_measured.append(dir_contents[i])

dir_contents = needs_to_measured 

In [0]:
# Function to check if any column contains 'Y'
def any_column_contains_Y(*args):
    return 'Y' in args

In [0]:
my_schema = t.StructType([
    t.StructField("EHHN", t.StringType(), True),
    t.StructField("SEGMENT", t.StringType(), True),
    t.StructField("SEGMENTATION", t.StringType(), True),
    t.StructField("FRONTEND_NAME", t.StringType(), True),
    t.StructField("PROPENSITY", t.StringType(), True),
    t.StructField("SEGMENT_TYPE", t.StringType(), True),
    t.StructField("PERCENTILE_SEGMENT", t.StringType(), True),
    t.StructField("PERIOD", t.StringType(), True),
    t.StructField("TAGS", t.StringType(), True)
])
df = spark.createDataFrame([], schema=my_schema)
my_schema_elig = t.StructType([
    t.StructField("EHHN", t.StringType(), True),
    t.StructField("SEGMENT", t.StringType(), True),
    t.StructField("SEGMENTATION", t.StringType(), True),
    t.StructField("FRONTEND_NAME", t.StringType(), True),
    t.StructField("PROPENSITY", t.StringType(), True),
    t.StructField("SEGMENT_TYPE", t.StringType(), True),
    t.StructField("PERCENTILE_SEGMENT", t.StringType(), True),
    t.StructField("PERIOD", t.StringType(), True),
    t.StructField("TAGS", t.StringType(), True),
    t.StructField("ONSITE_COUNT", t.IntegerType(), True),
    t.StructField("OFFSITE_COUNT", t.IntegerType(), True),
    t.StructField("OVERALL_ELIGIBILITY", t.BooleanType(), True),
])
combined_segmentation_df = spark.createDataFrame([], schema=my_schema_elig)
leftover_segment = {}
for i in dir_contents:
    for segmentations in segmentation_name:
          try:
            segment = tox_con.segmentation(segmentations)
            frontend_name = segment.frontend_name
            propensities = segment.propensities
            final_propensity = "".join(propensities)
            tag = segment.tags
            final_tag = "".join(tag)
            type_of_segment = segment.segment_type
            percentile_of_segment = segment.type
            latest_file = i
            rec = spark.read.csv(latest_file, header=True)
            path = rec.filter(f.col('audience')== segmentations).select('filepath').distinct().collect()[0][0]
            period = latest_file[-12:-4]
            segment_file = spark.read.format("delta").load(path)

            segment_file = (
                  segment_file.withColumn("SEGMENTATION", f.lit(segmentations))
                  .withColumn('FRONTEND_NAME', f.lit(frontend_name))
                  .withColumn("PROPENSITY", f.lit(final_propensity))
                  .withColumn("SEGMENT_TYPE", f.lit(type_of_segment))
                  .withColumn("PERCENTILE_SEGMENT", f.lit(percentile_of_segment))
                  .withColumn("PERIOD", f.lit(period))
                  .withColumn("TAGS", f.lit(final_tag))
            
            )
            
            segment_file = segment_file.filter(f.col("SEGMENT").isin(segment.propensities))
            segment_file = segment_file.select("EHHN", "SEGMENT", "SEGMENTATION", "FRONTEND_NAME",
                                                "PROPENSITY", "SEGMENT_TYPE", "PERCENTILE_SEGMENT","PERIOD","TAGS")
            df = df.union(segment_file)
            eligibility_analytical_dataset = df.join(latest_eligibility_df, on="ehhn", how='left')

            ### overall logic:

            column_names = ['NATIVE_ELIGIBLE_FLAG','TDC_ELIGIBLE_FLAG','SSE_ELIGIBLE_FLAG','FFD_ELIGIBLE_FLAG','EMAIL_ELIGIBLE_FLAG','PUSH_FLAG',
                            'FACEBOOK_FLAG','PANDORA_FLAG','CHICORY_FLAG','PUSH_FLAG','PREROLL_VIDEO_ELIGIBLE_FLAG','PINTEREST_ELIGIBLE_FLAG','ROKU_FLAG']

            # Register UDF
            contains_Y_udf = udf(any_column_contains_Y, BooleanType())

            # Apply UDF to create a new column


            eligibility_with_on_off_overall_flag = eligibility_analytical_dataset.withColumn('ONSITE_COUNT', f.when(((f.col('TDC_ELIGIBLE_FLAG') == 'Y') | (f.col('SSE_ELIGIBLE_FLAG') == 'Y') | (f.col('FFD_ELIGIBLE_FLAG') == 'Y') | (f.col('SS_ELIGIBLE_FLAG') == 'Y') | (f.col('PUSH_FLAG') == 'Y')), '1').otherwise('0').cast('integer'))\
                            .withColumn('OFFSITE_COUNT', f.when(((f.col('FACEBOOK_FLAG') == 'Y') | (f.col('LIVERAMP_FLAG') == 'Y') | (f.col('PANDORA_FLAG') == 'Y') | (f.col('PINTEREST_ELIGIBLE_FLAG') =='Y') | (f.col('PREROLL_VIDEO_ELIGIBLE_FLAG') == 'Y') | (f.col('CBA_ELIGIBLE_FLAG') == 'Y')), '1').otherwise('0').cast('integer'))\
                            .withColumn("OVERALL_ELIGIBILITY", contains_Y_udf(*[col(column) for column in column_names]))

            secondary_page_table = (eligibility_with_on_off_overall_flag.select("EHHN","SEGMENT","SEGMENTATION","FRONTEND_NAME","PROPENSITY","SEGMENT_TYPE","PERCENTILE_SEGMENT","PERIOD","TAGS","ONSITE_COUNT","OFFSITE_COUNT","OVERALL_ELIGIBILITY"))
                            
          
          except Exception as e:
                leftover_segment[segmentations] = (str(e) + str(dir_contents))
                pass

    combined_segmentation_df = combined_segmentation_df.union(secondary_page_table) #Get all segments combined
    



In [0]:
leftover_segment

In [0]:
segmentation_count_df = (combined_segmentation_df.groupBy('SEGMENTATION', 'PERIOD')
                        .agg(f.count('EHHN').alias("TOTAL_HH"),
                        f.max("frontend_name").alias("FRONTEND_NAME"),
                        f.max("SEGMENT").alias("SEGMENT"),
                        f.max("TAGS").alias("TAGS"),
                        f.max("PROPENSITY").alias("PROPENSITY"),
                        f.max("SEGMENT_TYPE").alias("SEGMENT_TYPE"),
                        f.max("PERCENTILE_SEGMENT").alias("PERCENTILE_SEGMENT"),
                        )
                        .select("SEGMENTATION","PERIOD","TOTAL_HH","FRONTEND_NAME","SEGMENT","TAGS","PROPENSITY","SEGMENT_TYPE","PERCENTILE_SEGMENT")
                        
                  ).cache()
                  
segmentation_onsite_offsite_count = (combined_segmentation_df.groupBy('SEGMENTATION', 'PERIOD')
                        .agg(f.count('EHHN').alias("TOTAL_HH"),
                        f.max("frontend_name").alias("FRONTEND_NAME"),
                        f.max("SEGMENT").alias("SEGMENT"),
                        f.max("PROPENSITY").alias("PROPENSITY"),
                        f.max("SEGMENT_TYPE").alias("SEGMENT_TYPE"),
                        f.max("PERCENTILE_SEGMENT").alias("PERCENTILE_SEGMENT"),
                        f.sum('ONSITE_COUNT').alias('ONSITE_COUNTS'),
                        f.sum('OFFSITE_COUNT').alias('OFFSITE_COUNTS'),
                        f.max("TAGS").alias("TAGS")
                        )
                        .select("SEGMENTATION", "PERIOD", "TOTAL_HH", "FRONTEND_NAME", "SEGMENT", "PROPENSITY", "SEGMENT_TYPE", "PERCENTILE_SEGMENT", "ONSITE_COUNTS", "OFFSITE_COUNTS", "TAGS")
                        
                    ).cache()

In [0]:
if needs_to_measured != []:
  segmentation_count_df.write.partitionBy('PERIOD').mode('append').parquet(f"{hh_location}")
  segmentation_onsite_offsite_count.write.partitionBy('PERIOD').mode('append').parquet(f"{elig_file}")

In [0]:
import subprocess
import resources.config as res_config
from functools import reduce
from effodata import ACDS, golden_rules, Joiner, Sifter, Equality, join_on
from kpi_metrics import KPI, AliasMetric, CustomMetric, AliasGroupby
from IPython.display import FileLink 
import sys
from typing import Union
import datetime as dt
from datetime import timedelta, datetime, date
from PIL import Image
import requests
import io
import pandas as pd
import pyspark.sql.functions as f
from pyspark.sql.window import Window
import commodity_segmentations.config as con
import matplotlib.pyplot as plt
import random
import resources.config as config
import seaborn as sns
import os

segmentation_list = get_all_segmentation_name()
acds = ACDS(use_sample_mart = False)

In [0]:
hh_count_df = spark.read.parquet(f"{hh_location}").filter(f.col("TAGS").isin('Alcohol'))
alc_segment = hh_count_df.select("SEGMENTATION").distinct().rdd.flatMap(lambda x: x).collect()
valid_image = spark.read.format("delta").load(image_location)

srt_dt = (date.today() - timedelta(days=385)).strftime("%Y%m%d")
end_dt = (date.today() - timedelta(days=10)).strftime("%Y%m%d")
srt_dt, end_dt

In [0]:

my_schema_elig = t.StructType([
    t.StructField("UPC", t.StringType(), True),
    t.StructField("URL", t.StringType(), True),
    t.StructField("SEGMENTATION", t.StringType(), True),
])
final_df = spark.createDataFrame([], schema=my_schema_elig)
failed_seg = {}

transaction = acds.get_transactions(start_date=srt_dt, end_date=end_dt)
gtins = (transaction.groupBy('gtin_no')
        .agg(f.count('transaction_code').alias('Total Redemers'),
            f.count('ehhn').alias("Total Households"),
            f.countDistinct('ehhn').alias("Total Distinct Counts"))
        .sort('Total Redemers', ascending=False)
        .cache())

for i in segmentation_list: 
        try: 
            if i not in alc_segment:
                seg_dir = f"{con.output_fp}upc_lists/{i}/"
                dir_contents = dbutils.fs.ls(seg_dir)
                dir_contents = [x[0] for x in dir_contents if i in x[1]]
                dir_contents.sort()
                seg_fp = dir_contents[-1]
                py_df = spark.read.format("delta").load(seg_fp)
            else:
                pim_fp = res_config.get_latest_modified_directory(res_config.azure_pim_core_by_cycle)
                pim = res_config.spark.read.parquet(pim_fp)
                pim = pim.select(
                f.col("upc").alias('upc_no'),
                f.col('familyTree.commodity').alias('commodity'),
                f.col('familyTree.subCommodity').alias('subcommodity')
                )

                pim = (pim.withColumn("commodity", f.col('commodity').getItem('name'))
                    .withColumn('subcommodity', f.col('subcommodity').getItem('name'))
                    .na.drop(subset=['subcommodity'])
                )
                alco_dict = tox_con.ss_dict[i]
                alc_comes = alco_dict["commodities"]
                alc_sucomes = alco_dict["sub_commodities"]
                py_df = pim.filter(f.col("commodity").isin(alc_comes) | f.col("subcommodity").isin(alc_sucomes)).select('upc_no').withColumnRenamed('upc_no','gtin_no'). dropDuplicates()

            getting_joined = py_df.join(gtins, on='gtin_no', how='left').sort('Total Redemers', ascending=False)
            df = getting_joined.join(valid_image, on='gtin_no', how='inner').sort('Total Redemers', ascending=False).select('gtin_no','URL').limit(10)
            upc_file = (df.withColumn("SEGMENTATION", f.lit(i)))
            final_df = final_df.union(upc_file)

        except Exception as e:
            failed_seg[i] = str(e)       


In [0]:
final_df.write.partitionBy('SEGMENTATION').mode("overwrite").parquet(f"{image_upc}")

In [0]:
# pim_fp = res_config.get_latest_modified_directory(res_config.azure_pim_core_by_cycle)
# pim = res_config.spark.read.parquet(pim_fp)
# pim = pim.select(
#   f.col("upc").alias('upc_no'),
#   f.col('familyTree.commodity').alias('commodity'),
#   f.col('familyTree.subCommodity').alias('subcommodity')
# )

# pim = (pim.withColumn("commodity", f.col('commodity').getItem('name'))
#        .withColumn('subcommodity', f.col('subcommodity').getItem('name'))
#        .na.drop(subset=['subcommodity'])
# )

# df = spark.read.parquet("abfss://media@sa8451dbxadhocprd.dfs.core.windows.net/audience_factory/powerbi_inputs/household_counts/").filter(f.col("TAGS").isin('Alcohol'))
# alc_segment = df.select("SEGMENTATION").distinct().rdd.flatMap(lambda x: x).collect()

# alco_dict = con.ss_dict[alc_segment[0]]
# alc_comes = alco_dict["commodities"]
# alc_sucomes = alco_dict["sub_commodities"]

# upcs = pim.filter(f.col("commodity").isin(alc_comes) | f.col("subcommodity").isin(alc_sucomes)).select('upc_no').withColumnRenamed('upc_no','gtin_no').dropDuplicates()
# # alc_upcs = upcs.select("upc_no").rdd.flatMap(lambda x: x).collect()

# transaction = acds.get_transactions(start_date=srt_dt, end_date=end_dt)
# gtins = (transaction.groupBy('gtin_no')
#         .agg(f.count('transaction_code').alias('Total Redemers'),
#             f.count('ehhn').alias("Total Households"),
#             f.countDistinct('ehhn').alias("Total Distinct Counts"))
#         .sort('Total Redemers', ascending=False))
# getting_joined = upcs.join(gtins, on='gtin_no', how='left').sort('Total Redemers', ascending=False).select('gtin_no')
# n = 10
# image_upcs, upc_url = draw_nonempty_upcs(getting_joined, n)

# # Combine the lists using zip
# combined_data = zip(image_upcs, upc_url)

# # Create a DataFrame with column names
# df = spark.createDataFrame(combined_data, ["UPC", "URL"])
# upc_file = (df.withColumn("SEGMENTATION", f.lit(alc_segment[0])))

# # final_df = final_df.union(upc_file)

In [0]:
display(final_df)